# 1. Подготовка данных

In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from math import sqrt
import timeit
import numpy as np

In [6]:
data = pd.read_csv('/datasets/autos.csv')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB
None


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [7]:
#посмотрим на данные
for column in ['Price',
               'VehicleType',
               'RegistrationYear',
               'Gearbox',
               'Power',
               'Kilometer',
               'RegistrationMonth',
               'FuelType',
               'Brand',
               'NotRepaired',
               'NumberOfPictures']:
    pt = data.pivot_table(index=column, values='LastSeen', aggfunc='count')
    display(pt)

,LastSeen
Price,
0,10772
1,1189
2,12
3,8
4,1
...,...
19995,10
19997,1
19998,6


,LastSeen
VehicleType,
bus,28775
convertible,20203
coupe,16163
other,3288
sedan,91457
small,79831
suv,11996
wagon,65166


,LastSeen
RegistrationYear,
1000,37
1001,1
1039,1
1111,3
1200,1
...,...
9000,3
9229,1
9450,1


,LastSeen
Gearbox,
auto,66285
manual,268251


,LastSeen
Power,
0,40225
1,34
2,10
3,9
4,30
...,...
17932,1
19208,1
19211,1


...

,LastSeen
RegistrationMonth,
0,37352
1,23219
2,21267
3,34373
4,29270
5,29153
6,31508
7,27213
8,22627


,LastSeen
FuelType,
cng,565
electric,90
gasoline,98720
hybrid,233
lpg,5310
other,204
petrol,216352


,LastSeen
Brand,
alfa_romeo,2314
audi,29456
bmw,36914
chevrolet,1754
chrysler,1439
citroen,5148
dacia,900
daewoo,542
daihatsu,806


,LastSeen
NotRepaired,
no,247161
yes,36054


,LastSeen
NumberOfPictures,
0,354369


Необходима предобработка данных.
- DateCrawled - удаляем (не влияет на цену)
- Price - удаляем цены ниже 50 евро (машина не может стоить меньше металлолома)
- VehicleType - ставим 'other'
- RegistrationYear - удаляем всё, что меньше 1950 и больше 2020
- Gearbox - ставим 'manual', т.к. таких коробок намного больше
- Power - пропуски заполняем по медиане для модели. После этого удаляем всё, что меньше 40 л.с. и больше 1000 л.с. (чтобы не отсеять суперкары)
- Model - удаляем столбец (он не настолько важен, чтобы получить проблемы с дамми-ловушками)
- RegistrationMonth - удаляем (у нас есть год регистрации, который напрямую указывает на год выпуска а/м. Месяц для этого не нужен)
- FuelType - ставим petrol как самый популярный
- Brand - так же дропаем, аналогично Model
- NotRepaired - ставим yes, т.к. пункт могли пропустить, если не было ремонта
- NumberOfPictures - можно удалить, т.к. везде 0
- DateCreated - удаляем (не влияет на цену)
- PostalCode - оставим, т.к. на цену влияет регион
- LastSeen - удаляем

In [8]:
data_cars = data #чтобы не импортировать данные заново при ошибке

#удаление цен ниже 50 евро
data_cars = data_cars.loc[data_cars['Price'] >= 50]

#заполнение VehicleType
data_cars['VehicleType'] = data_cars['VehicleType'].fillna('other')

#удаление некорректных годов регистрации
data_cars = data_cars.loc[(data_cars['RegistrationYear'] >= 1950) & (data_cars['RegistrationYear'] <= 2020)]

#заполнение поля Gearbox
data_cars['Gearbox'] = data_cars['Gearbox'].fillna('manual')

#заполнение пропусков в Power
model_list = data_cars.loc[data_cars['Power'] == 0, 'Model'].drop_duplicates()
for model in model_list:
    model_power = data_cars.loc[(data_cars['Model'] == model) & (data_cars['Power'] > 0), 'Power'].median()
    data_cars.loc[data_cars['Model'] == model, 'Power'] = data_cars.loc[data_cars['Model'] == model, 'Power'].fillna(model_power)

data_cars = data_cars.loc[(data_cars['Power'] >= 40) & (data_cars['Power'] <= 1000)]

#удаление Model, RegistrationMonth, Brand, DateCreated и LastSeen
data_cars = data_cars.drop(['DateCrawled',
                            'Model',
                            'RegistrationMonth',
                            'Brand',
                            'NumberOfPictures',
                            'DateCreated',
                            'LastSeen'], axis=1)

#заполнение FuelType
data_cars['FuelType'] = data_cars['FuelType'].fillna('petrol')

#заполнение NotRepaired
data_cars['NotRepaired'] = data_cars['NotRepaired'].fillna('yes')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
print(data_cars.info())
data_cars.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304744 entries, 1 to 354368
Data columns (total 9 columns):
Price               304744 non-null int64
VehicleType         304744 non-null object
RegistrationYear    304744 non-null int64
Gearbox             304744 non-null object
Power               304744 non-null int64
Kilometer           304744 non-null int64
FuelType            304744 non-null object
NotRepaired         304744 non-null object
PostalCode          304744 non-null int64
dtypes: int64(5), object(4)
memory usage: 23.3+ MB
None


,Price,VehicleType,RegistrationYear,Gearbox,Power,Kilometer,FuelType,NotRepaired,PostalCode
1,18300,coupe,2011,manual,190,125000,gasoline,yes,66954
2,9800,suv,2004,auto,163,125000,gasoline,yes,90480
3,1500,small,2001,manual,75,150000,petrol,no,91074
4,3600,small,2008,manual,69,90000,gasoline,no,60437
5,650,sedan,1995,manual,102,150000,petrol,yes,33775


# 2. Обучение моделей

In [13]:
#dummy
data_prepared = pd.get_dummies(data_cars, drop_first=True)
data_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304744 entries, 1 to 354368
Data columns (total 20 columns):
Price                      304744 non-null int64
RegistrationYear           304744 non-null int64
Power                      304744 non-null int64
Kilometer                  304744 non-null int64
PostalCode                 304744 non-null int64
VehicleType_convertible    304744 non-null uint8
VehicleType_coupe          304744 non-null uint8
VehicleType_other          304744 non-null uint8
VehicleType_sedan          304744 non-null uint8
VehicleType_small          304744 non-null uint8
VehicleType_suv            304744 non-null uint8
VehicleType_wagon          304744 non-null uint8
Gearbox_manual             304744 non-null uint8
FuelType_electric          304744 non-null uint8
FuelType_gasoline          304744 non-null uint8
FuelType_hybrid            304744 non-null uint8
FuelType_lpg               304744 non-null uint8
FuelType_other             304744 non-null uint8
FuelTyp

In [14]:
data_prepared.head(3)

,Price,RegistrationYear,Power,Kilometer,PostalCode,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,NotRepaired_yes
1,18300,2011,190,125000,66954,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1
2,9800,2004,163,125000,90480,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
3,1500,2001,75,150000,91074,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0


In [15]:
#разбивка на признаки и целевой признак
features = data_prepared.drop('Price', axis=1)
target = data_prepared['Price']
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target,
                                                                              test_size=0.25,
                                                                              random_state=12345)

In [16]:
#три модели градиентного бустинга - линейная регрессия, случайный лес и LGBMRegressor
model1 = LinearRegression()
model2 = RandomForestRegressor(max_depth = 50, n_estimators = 80, random_state=12345)
model3 = LGBMRegressor(max_depth=60, n_estimators=100, learning_rate=0.7, random_state=12345)

In [9]:
#обучение и расчет качества для каждой модели
i = 1
models = [model1, model2, model3]
for model in models:
    start = timeit.default_timer() #потому что %%time выведет время выполнения всей ячейки
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    end = timeit.default_timer()
    rmse = sqrt(mean_squared_error(target_valid, predictions))
    print('RMSE модели ', i, ':', rmse)
    print('Время работы модели ', i, ', с:', end - start)
    i += 1

RMSE модели  1 : 2860.5474449759404
Время работы модели  1 , с: 0.4086904972791672
RMSE модели  2 : 1787.97732778227
Время работы модели  2 , с: 100.54622570704669
RMSE модели  3 : 1846.4391796256905
Время работы модели  3 , с: 77.94648784957826


# 3. Анализ моделей

В рамках проекта построены 3 модели машинного обучения: линейная регрессия, случайный лес и градиентный бустинг. Выводы:
- Наименьшую точность имеет линейная регрессия с гиперпараметрами по умолчанию. При этом у неё наименьшее время работы.
- Случайный лес имеет наибольшую точность, при этом работает значительно дольше других моделей.
- Градиентный бустинг имеет чуть худшую точность по сравнению со случайным лесом, однако работает быстрее. Плюсом этой модели также является большое количество доступных гиперпараметров, что позволяет более тонко настраивать модель.